<a href="https://colab.research.google.com/github/ashishrai7/temp_ash/blob/main/micrograd_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# micrograd exercises

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

## section 1: derivatives

In [21]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [22]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):

  df_da = -3*a**2 - 0.5* a**(-0.5)
  df_db = 3*cos(3*b) + 2.5*b**1.5
  df_dc = c**(-2)
  return [df_da, df_db, df_dc]
  # return [0, 0, 0] # todo, return [df/da, df/db, df/dc]

# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [23]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

# -----------
numerical_grad = [0, 0, 0] # TODO
# -----------
h = 0.0000001
a,b,c = (2,3,4)
f(2, 3, 4)
f_zeor = f(a,b,c)
fa = f(a+h,b,c)
fb = f(a,b+h,c)
fc = f(a,b,c+h)
df_da = (fa - f_zeor)/h
df_db = (fb - f_zeor)/h
df_dc = (fc - f_zeor)/h

numerical_grad = [df_da, df_db, df_dc]


for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553966448771
OK for dim 1: expected 10.25699027111255, yours returns 10.256990385926201
OK for dim 2: expected 0.0625, yours returns 0.06249999628948899


In [24]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# -----------
numerical_grad2 = [0, 0, 0] # TODO
fap = f(a+h,b,c)
fbp = f(a,b+h,c)
fcp = f(a,b,c+h)

fam = f(a-h,b,c)
fbm = f(a,b-h,c)
fcm = f(a,b,c-h)

df_da = (fap - fam)/(2*h)
df_db = (fbp - fbm)/(2*h)
df_dc = (fcp - fcm)/(2*h)

numerical_grad2 = [df_da, df_db, df_dc]

# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553380251014
OK for dim 1: expected 10.25699027111255, yours returns 10.256990252699438
OK for dim 2: expected 0.0625, yours returns 0.06250000073038109


## section 2: support for softmax

In [102]:
# Value class starter code, with many functions taken out
from math import exp, log

class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out

  # ------
  # re-implement all the other functions needed for the exercises below
  # your code here
  # TODO

  def __rmul__(self,other):
    return self*other
  def __radd__(self,other):
    return self + other

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self,other), _op='*')
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out.backward = _backward
    return out

  def __pow__(self,other):
    assert isinstance(other, (int, float)) , 'only accept int and float powers for now'
    out = Value(self.data**other, _children=(self,), _op= f'**{other}')
    def _backward():
      self.grad += other * (self.data**(other -1)) * out.grad
    out._backward = _backward
    return out

  def __neg__(self):
    return self * -1

  def __sub__(self, other):
    return self + (-other)

  def __truediv__(self, other):
    return self*(other**-1)

  def tanh(self):
    e = exp(2*self.data)
    t =(e-1)/(e+1)
    out = Value(t, (self,), _op='tanh')
    def _backward():
      self.grad = ( 1- t**2) * out.grad
    out._backward = _backward
    return out

  def log(self):
    out = Value(log(self.data), (self,), _op='log')
    def _backward():
      self.grad = 1*self.data**(-1) * out.grad
    out._backward = _backward
    return out


  def exp(self):
    out = Value(exp(self.data), (self,), _op='exp')
    def _backward():
      self.grad+= out.data * out.grad
    out._backward = _backward
    return out
  # ------

  def backward(self): # exactly as in video
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [103]:

denom = sum([logit.exp() for logit in logits])
probs = [ t.exp()/denom for t in logits]
probs

[Value(data=0.04177257051535045),
 Value(data=0.839024507462532),
 Value(data=0.00565330266221633),
 Value(data=0.11354961935990122)]

In [104]:
loss = -probs[3].log()

loss.backward()
loss.grad

In [139]:
from  math import log , exp
class Value:

    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self._prev = set(_children)
        self.grad = 0.0   # gradiant value at the backpropagation time
        self._backward = lambda : None
        self._op = _op
        self.label= label

    def __repr__(self):
        return f"Value(data={self.data})"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), _op='+')

        def _backward():
            self.grad += 1.0* out.grad
            other.grad += 1.0* out.grad
        out._backward = _backward
        return out
    def __rmul__(self, other) : # other*self
        return self*other

    def __radd__(self, other): # other + self
        return self + other

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), _op='*')

        def _backward():
            self.grad += other.data*out.grad
            other.grad += self.data*out.grad
        out._backward = _backward
        return out
    def __pow__(self, other):
        assert isinstance(other,(int, float)), "only support int/float powers for now"
        out = Value(self.data**other, _children=(self,), _op= f'**{other}')
        def _backward():
            self.grad += other * (self.data ** (other-1)) * out.grad
        out._backward = _backward
        return out

    def __neg__(self): # -self
        return self * -1

    def __sub__(self, other): # self - other
        return self + (-other)


    def __truediv__(self,other):
        return self*other**-1



    def tanh(self):
        x = self.data
        t = (exp(2*x) -1)/(exp(2*x) +1)
        out = Value(t, _children=(self,), _op='tanh')

        def _backward():
            self.grad += (1 - t**2)*out.grad
        out._backward = _backward
        return out

    def exp(self):
        x = self.data
        out = Value(exp(x),_children=(self,), _op='exp')
        def _backward():
            self.grad+= out.data*out.grad
        out._backward = _backward
        return out

    def log(self):
      out = Value(log(self.data), (self,), _op='log')
      def _backward():
        self.grad = 1*self.data**(-1) * out.grad
      out._backward = _backward
      return out


    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        self.grad =1

        for node in reversed(topo):
            node._backward()



In [140]:
a = Value(2)
b = Value(3)

c = a * b
c

Value(data=6)

In [141]:

print(a, b, c)
print(a.grad, b.grad, c.grad)
c.backward()
print(a.grad, b.grad, c.grad)

Value(data=2) Value(data=3) Value(data=6)
0.0 0.0 0.0
3.0 2.0 1


In [122]:
c.backward()
c.grad

0.0

1.0 0.0 1.0


In [142]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example

print(probs)
print(logits)
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


[Value(data=0.04177257051535045), Value(data=0.839024507462532), Value(data=0.00565330266221633), Value(data=0.11354961935990122)]
[Value(data=0.0), Value(data=3.0), Value(data=-2.0), Value(data=1.0)]
2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [143]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch
